In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams = mpl.rc_params_from_file("includes/mlprc")

In [3]:
import numpy as np

from includes import utils
from includes import config
from includes import algorithms

In [4]:
def evaluate_algorithm(train_set, test_set, algorithm, verbose, *args):
    if len(test_set) == 0:
        np.random.shuffle(train_set)
        
        train_set, test_set = np.split(train_set, [int(train_set.shape[0] * 0.7)])
        
    predicted, model = algorithm(train_set, test_set, verbose, *args)

    accuracy = np.mean(test_set[:, -1] == predicted) * 100

    return accuracy, model

In [5]:
train_set, test_set, n_classes = utils.get_data("iris")

In [6]:
conf = config.Config(verbose=False)

In [7]:
conf.set_algorithm("sft-reg")
accuracy, model = evaluate_algorithm(
    train_set,
    test_set,
    algorithms.softmax_regression,
    conf.verbose,
    conf.lr,
    n_classes,
    conf.n_epochs,
    conf.batch_size,
    conf.tol
)

print "Softmax Regression"
print "Accuracy: %s" % accuracy

Softmax Regression
Accuracy: 89.23076923076924


In [8]:
conf.set_algorithm("boosted-sft-reg")

boosted_accuracies = []
boosted_models = []
for activation in ["logistic", "relu", "tanh", "identity"]:
    accuracy, model = evaluate_algorithm(
        train_set,
        test_set,
        algorithms.boosted_softmax_regression,
        conf.verbose,
        conf.lr,
        n_classes,
        conf.n_epochs,
        conf.batch_size,
        conf.m_stop,
        conf.eta,
        activation,
        "plots/sft-data-%s.npy" % activation
    )
    
    boosted_models.append(model)
    boosted_accuracies.append(accuracy)

    print "Boosted Softmax Regression, Activation = %s" % activation
    print "Final Test Accuracy: %f" % accuracy

Boosted Softmax Regression, Activation = logistic
Final Test Accuracy: 92.307692
Boosted Softmax Regression, Activation = relu
Final Test Accuracy: 76.153846
Boosted Softmax Regression, Activation = tanh
Final Test Accuracy: 96.923077
Boosted Softmax Regression, Activation = identity
Final Test Accuracy: 97.692308


In [9]:
conf.set_algorithm("multi-mlp")

pretrained_mlp_models = []
pretrained_mlp_accuracies = []

activations = ["logistic", "relu", "tanh", "identity"]
for index, activation in enumerate(activations):
    accuracy, model = evaluate_algorithm(
        train_set,
        test_set,
        algorithms.pretrained_mlp,
        conf.verbose,
        conf.lr,
        n_classes,
        activation,
        conf.n_epochs,
        conf.batch_size,
        conf.tol,
        boosted_models[index]
    )
    
    pretrained_mlp_accuracies.append(accuracy)
    pretrained_mlp_models.append(model)

    print "Boosted + MLP, Activation = %s" % activation
    print "Final Test Accuracy: %f" % accuracy

ValueError: could not broadcast input array from shape (2) into shape (3)

In [11]:
conf.set_algorithm("multi-mlp")

mlp_models = []
mlp_accuracies = []

activations = ["logistic", "relu", "tanh", "identity"]
for activation in activations:
    accuracy, model = evaluate_algorithm(
        train_set,
        test_set,
        algorithms.mlp,
        conf.verbose,
        conf.lr,
        n_classes,
        activation,
        conf.n_epochs,
        conf.batch_size,
        conf.hidden_size,
        conf.tol / 10
    )
    
    mlp_accuracies.append(accuracy)
    mlp_models.append(model)

    print "Multi Layer Perceptron, Activation = %s" % activation
    print "Accuracy: %f" % accuracy

Multi Layer Perceptron, Activation = logistic
Accuracy: 38.461538
Multi Layer Perceptron, Activation = relu
Accuracy: 97.692308
Multi Layer Perceptron, Activation = tanh
Accuracy: 40.000000
Multi Layer Perceptron, Activation = identity
Accuracy: 38.461538


In [ ]:
for i in range(4):
    print "%.6f  %.6f  %.6f" % (boosted_accuracies[i], mlp_accuracies[i], pretrained_mlp_accuracies[i])

In [ ]:
colors = ["blue", "red", "green", "orange"]
activations = ["logistic", "relu", "tanh", "identity"]

for color, activation in zip(colors, activations):
    data = np.load("plots/sft-data-%s.npy" % activation)
    plt.plot(data, label=activation, color=color)
    
plt.xlabel("m_stop")
plt.ylabel("test accuracy")
    
plt.legend()
plt.tight_layout()
plt.savefig("plots/plot.png", transparent=True)
plt.show()